# MQTT PUBLISH 測試應用程式

這是一個測試用的 MQTT 發布應用程式，用於發送感測器數據到 MQTT Broker。

## 功能
- 連接到本地 MQTT Broker
- 發布溫度、濕度、電燈狀態等測試數據
- 支援批次發送多筆數據
- 顯示發送狀態和結果

## 使用說明
1. **按順序執行 Cell 1 → 2 → 3 → 4**
2. Cell 4（建立連線）只執行一次
3. 測試 Cells（5-8）可以重複執行
4. 完成後執行 Cell 9 清理連線


In [1]:
# Cell 1: 匯入必要的套件
import paho.mqtt.client as mqtt
import json
import time
from datetime import datetime
import random

print("✅ 套件匯入成功！")


✅ 套件匯入成功！


In [2]:
# Cell 2: MQTT 配置

# MQTT Broker 位址設定
BROKER = "localhost"  # 本機 Broker
# BROKER = "192.168.1.16"  # 或使用 Raspberry Pi 的 IP

PORT = 1883
TOPIC = "客廳/感測器"

# 客戶端 ID（加入時間戳記以避免重複）
CLIENT_ID = f"jupyter_publisher_{int(time.time())}"

print("=" * 60)
print("📡 MQTT 連線設定")
print("=" * 60)
print(f"   Broker: {BROKER}:{PORT}")
print(f"   Topic: {TOPIC}")
print(f"   Client ID: {CLIENT_ID}")
print("=" * 60)


📡 MQTT 連線設定
   Broker: localhost:1883
   Topic: 客廳/感測器
   Client ID: jupyter_publisher_1765561516


In [3]:
# Cell 3: 定義回調函數

def on_connect(client, userdata, flags, reason_code, properties):
    """連線成功時的回調"""
    if reason_code == 0:
        print("✅ 成功連接到 MQTT Broker")
    else:
        print(f"❌ 連線失敗，錯誤碼: {reason_code}")

def on_publish(client, userdata, mid, reason_code=None, properties=None):
    """訊息發布成功時的回調"""
    print(f"  ✓ 訊息已發布 (ID: {mid})")

def on_disconnect(client, userdata, flags, reason_code, properties):
    """斷線時的回調"""
    if reason_code == 0:
        print("👋 已正常斷開連線")
    else:
        print(f"⚠️ 意外斷線，錯誤碼: {reason_code}")

print("✅ 回調函數定義完成")


✅ 回調函數定義完成


In [4]:
# Cell 4: 建立連線（⚠️ 只執行一次！）

# 檢查是否已有連線
if 'client' in globals():
    print("⚠️ 警告：client 已存在！")
    print("   如需重新連線，請先執行 Cell 9 清理，然後重啟 Kernel")
else:
    try:
        # 建立新客戶端
        client = mqtt.Client(
            callback_api_version=mqtt.CallbackAPIVersion.VERSION2,
            client_id=CLIENT_ID,
            clean_session=True
        )
        
        # 設定回調函數
        client.on_connect = on_connect
        client.on_publish = on_publish
        client.on_disconnect = on_disconnect
        
        # 連線
        print(f"🔌 正在連接到 {BROKER}:{PORT}...")
        client.connect(BROKER, PORT, 60)
        
        # 啟動網路迴圈
        client.loop_start()
        
        # 等待連線
        time.sleep(1)
        
        print("\n" + "=" * 60)
        print("✅ MQTT 客戶端已就緒")
        print("=" * 60)
        print("💡 現在可以執行下方的發布測試 cells")
        
    except Exception as e:
        print(f"❌ 連線失敗: {e}")
        import traceback
        traceback.print_exc()


🔌 正在連接到 localhost:1883...
✅ 成功連接到 MQTT Broker

✅ MQTT 客戶端已就緒
💡 現在可以執行下方的發布測試 cells


In [8]:
# Cell 5: 生成測試數據

def generate_sensor_data(message_id=1):
    """生成模擬的感測器數據"""
    data = {
        "temperature": round(random.uniform(18, 30), 2),
        "humidity": round(random.uniform(40, 80), 2),
        "light_status": random.choice(["開", "關"]),
        "timestamp": datetime.now().isoformat(),
        "device": "Jupyter 測試裝置",
        "message_id": message_id
    }
    return data

# 測試
test_data = generate_sensor_data()
print("📊 測試數據範例：")
print(json.dumps(test_data, indent=2, ensure_ascii=False))


📊 測試數據範例：
{
  "temperature": 28.55,
  "humidity": 79.25,
  "light_status": "開",
  "timestamp": "2025-12-13T01:48:56.439504",
  "device": "Jupyter 測試裝置",
  "message_id": 1
}


In [7]:
# Cell 6: 發布單筆測試數據（可重複執行）

def publish_single_message():
    """發布一筆測試訊息"""
    if 'client' not in globals():
        print("❌ 錯誤：client 不存在，請先執行 Cell 4 建立連線")
        return
    
    try:
        data = generate_sensor_data()
        json_data = json.dumps(data, ensure_ascii=False)
        
        result = client.publish(TOPIC, json_data, qos=1)
        
        if result.rc == mqtt.MQTT_ERR_SUCCESS:
            print("✅ 發布成功！")
            print(f"   🌡️  溫度: {data['temperature']}°C")
            print(f"   💧 濕度: {data['humidity']}%")
            print(f"   💡 電燈: {data['light_status']}")
        else:
            print(f"❌ 發布失敗，錯誤碼: {result.rc}")
            
    except Exception as e:
        print(f"❌ 錯誤: {e}")

print("=" * 60)
print("📤 發布單筆測試數據")
print("=" * 60)
publish_single_message()


📤 發布單筆測試數據
✅ 發布成功！
   🌡️  溫度: 19.28°C
   💧 濕度: 77.51%
   💡 電燈: 關
  ✓ 訊息已發布 (ID: 1)


In [8]:
# Cell 7: 批次發布多筆數據（可重複執行）

def publish_multiple_messages(count=5, interval=1):
    """批次發布多筆測試訊息"""
    if 'client' not in globals():
        print("❌ 錯誤：client 不存在，請先執行 Cell 4 建立連線")
        return
    
    print("=" * 60)
    print(f"📤 批次發布 {count} 筆測試數據（間隔 {interval} 秒）")
    print("=" * 60)
    
    for i in range(1, count + 1):
        try:
            data = generate_sensor_data(i)
            json_data = json.dumps(data, ensure_ascii=False)
            
            print(f"\n[{i}/{count}] 發布數據:")
            print(f"  🌡️  溫度: {data['temperature']}°C")
            print(f"  💧 濕度: {data['humidity']}%")
            print(f"  💡 電燈: {data['light_status']}")
            
            result = client.publish(TOPIC, json_data, qos=1)
            
            if result.rc != mqtt.MQTT_ERR_SUCCESS:
                print(f"  ❌ 發布失敗")
            
            if i < count:
                time.sleep(interval)
                
        except Exception as e:
            print(f"[{i}/{count}] ❌ 錯誤: {e}")
    
    print("\n" + "=" * 60)
    print("✅ 批次發布完成")
    print("=" * 60)

# 執行批次發布
publish_multiple_messages(5, 1)


📤 批次發布 5 筆測試數據（間隔 1 秒）

[1/5] 發布數據:
  🌡️  溫度: 28.19°C
  💧 濕度: 58.56%
  💡 電燈: 開
  ✓ 訊息已發布 (ID: 2)

[2/5] 發布數據:
  🌡️  溫度: 22.84°C
  💧 濕度: 56.09%
  💡 電燈: 開
  ✓ 訊息已發布 (ID: 3)

[3/5] 發布數據:
  🌡️  溫度: 19.09°C
  💧 濕度: 41.67%
  💡 電燈: 開
  ✓ 訊息已發布 (ID: 4)

[4/5] 發布數據:
  🌡️  溫度: 28.71°C
  💧 濕度: 41.21%
  💡 電燈: 關
  ✓ 訊息已發布 (ID: 5)

[5/5] 發布數據:
  🌡️  溫度: 23.83°C
  💧 濕度: 72.94%
  💡 電燈: 關

✅ 批次發布完成
  ✓ 訊息已發布 (ID: 6)


In [9]:
# Cell 8: 發布自訂數據（可重複執行）

def publish_custom_data(temperature, humidity, light_status):
    """發布自訂的感測器數據"""
    if 'client' not in globals():
        print("❌ 錯誤：client 不存在，請先執行 Cell 4 建立連線")
        return
    
    try:
        data = {
            "temperature": temperature,
            "humidity": humidity,
            "light_status": light_status,
            "timestamp": datetime.now().isoformat(),
            "device": "Jupyter 自訂數據"
        }
        
        json_data = json.dumps(data, ensure_ascii=False)
        result = client.publish(TOPIC, json_data, qos=1)
        
        if result.rc == mqtt.MQTT_ERR_SUCCESS:
            print("✅ 自訂數據發布成功！")
            print(f"   🌡️  溫度: {temperature}°C")
            print(f"   💧 濕度: {humidity}%")
            print(f"   💡 電燈: {light_status}")
        else:
            print("❌ 發布失敗")
            
    except Exception as e:
        print(f"❌ 錯誤: {e}")

# 範例：發布自訂數據
print("=" * 60)
print("📤 發布自訂數據")
print("=" * 60)
publish_custom_data(25.5, 60.0, "開")


📤 發布自訂數據
✅ 自訂數據發布成功！
   🌡️  溫度: 25.5°C
   💧 濕度: 60.0%
   💡 電燈: 開
  ✓ 訊息已發布 (ID: 7)


In [10]:
# Cell 9: 清理連線（測試結束時執行）

def cleanup():
    """清理 MQTT 連線"""
    if 'client' in globals():
        try:
            print("🧹 正在關閉連線...")
            client.loop_stop()
            client.disconnect()
            print("✅ 連線已關閉")
        except Exception as e:
            print(f"⚠️ 清理時發生錯誤: {e}")
    else:
        print("ℹ️ 沒有需要清理的連線")

cleanup()


🧹 正在關閉連線...
✅ 連線已關閉
